In [1]:
import pandas as pd 
# %%
import openai
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_catalyse = pd.read_excel("../GPT35_head_100.xlsx")

In [7]:
df_nacres= pd.read_pickle(r"../data\NACRES_embedded.pkl")

In [12]:
df_nacres.columns

Index(['nacres.code', 'nacres.description.fr', 'nacres.description.en',
       'ceda.code', 'ceda.description', 'ceda.ef.kg.co2e.per.euro',
       'ceda.uncertainty.attr.kg.co2e.per.euro',
       'ceda.uncertainty.80pct.kg.co2e.per.euro', 'ademe.code',
       'ademe.description', 'ademe.ef.kg.co2e.per.euro',
       'ademe.uncertainty.attr.kg.co2e.per.euro',
       'ademe.uncertainty.80pct.kg.co2e.per.euro', 'useeio.code',
       'useeio.description', 'useeio.ef.kg.co2e.per.euro',
       'useeio.uncertainty.attr.kg.co2e.per.euro',
       'useeio.uncertainty.80pct.kg.co2e.per.euro',
       'per1p5macro.ef.kg.co2e.per.euro',
       'per1p5macro.uncertainty.attr.kg.co2e.per.euro',
       'per1p5macro.uncertainty.80pct.kg.co2e.per.euro', 'meso.code',
       'meso.description', 'meso.ef.kg.co2e.per.euro',
       'meso.uncertainty.kg.co2e.per.euro', 'micro.code', 'micro.description',
       'micro.ef.kg.co2e.per.euro', 'micro.uncertainty.kg.co2e.per.euro',
       'method', 'per1p5.ef.kg.co2e.

In [6]:
df_catalyse["Chosen_NACRES_name"]

0      OTHER COMPUTER PERIPHERALS (MONITORS, KEYBOARD...
1                                               KEROSENE
2            PRIMARY ANTIBODIES PAIRED WITH CELL MARKERS
3            BACTERIOLOGY : SPECIFIC PLASTIC CONSUMABLES
4       BIOMOL: GEL AND MEMBRANE IMAGERS AND ACCESSORIES
                             ...                        
96     BIOLOGY: AUTOMATIC OLIGONUCLEOTIDE SYNTHESIS M...
97              CYTOKINES, GROWTH FACTORS AND INHIBITORS
98               GLASS AND PLASTIC SLIDES AND COVERSLIPS
99               GLASS AND PLASTIC SLIDES AND COVERSLIPS
100    TIPS (CONES) FOR SINGLE AND MULTICHANNEL MICRO...
Name: Chosen_NACRES_name, Length: 101, dtype: object

In [10]:
#Merged df_catalyse with df_nacres based on "Chosen_NACRES_name" left and nacres.description.en on right
df_catalyse_nacres = pd.merge(df_catalyse, df_nacres, left_on="Chosen_NACRES_name", right_on="nacres.description.en", how="left")

In [16]:
df_catalyse_nacres["Total_CO2 kg"] =  df_catalyse_nacres["ademe.ef.kg.co2e.per.euro"]*df_catalyse_nacres["Prix total"]

In [17]:
df_catalyse_nacres.to_excel("GPT35_head_100_CO2.xlsx")

In [5]:
from openai import OpenAI
import json
import os
with open(r"C:\Users\cruz" + r'\API_openAI.txt', 'r') as f:
    read_api_key = f.readline()
os.environ["OPENAI_API_KEY"] = read_api_key
f = os.environ["OPENAI_API_KEY"]

client = OpenAI(
    api_key =f
)

example_json = {
  "ski_resorts": [
    {
      "name": "Les Portes du Soleil",
      "slope_kilometers": 600
    }
  ]
}

prompt = "Provide valid JSON output. Provide the top 10 largest ski resorst in Europe. Rankin them on slope kilometers (descending) Provide one column 'name' and a column 'slope_kilometers' representing the total slope kilometers"

chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={"type":"json_object"},
    messages=[
        {"role":"system","content":"""Provide output in valid JSON. Your are given a dictionnary in json format.
                For each elements, you need to choose between 1 of the options in the 'Options' field.
                The data schema should be like this :""" + json.dumps(example_json)},
        {"role":"user","content":prompt}
    ]
)

finish_reason = chat_completion.choices[0].finish_reason

if(finish_reason == "stop"):
    data = chat_completion.choices[0].message.content

    ski_resorts = json.loads(data)

    for ski_resort in ski_resorts['ski_resorts']:
        print(ski_resort['name']+" : "+str(ski_resort['slope_kilometers'])+"km")
else :
    print("Error! provide more tokens please")

Les Portes du Soleil : 600km
Les Trois Vallees : 600km
Paradiski : 425km
Via Lattea : 400km
Sella Ronda : 400km
Espace Killy : 300km
Silvretta Arena : 238km
Arlberg : 305km
Serre Chevalier : 250km
Zermatt-Cervinia : 360km


In [7]:
messages=[
    {"role":"system","content":"""Provide output in valid JSON. Your are given a dictionnary in json format.
            For each elements, you need to choose between 1 of the options in the 'Options' field.
            The data schema should be like this :""" + json.dumps(example_json)},
    {"role":"user","content":prompt}
]

In [9]:
messages[0]

{'role': 'system',
 'content': 'Provide output in valid JSON. Your are given a dictionnary in json format.\n            For each elements, you need to choose between 1 of the options in the \'Options\' field.\n            The data schema should be like this :{"ski_resorts": [{"name": "Les Portes du Soleil", "slope_kilometers": 600}]}'}

In [11]:
import pandas as pd

In [12]:
df_gpt = pd.read_excel(r"../data\achats_GVG_NACRES_GPT.xlsx")

In [13]:
df_source= pd.read_pickle(r"../data\Results\achats_GVG_lab_2022.pkl")

In [14]:
#Merge both dataframes on index
df_gpt_source = pd.merge(df_gpt, df_source, left_index=True, right_index=True)

In [17]:
df_NACRES = pd.read_pickle(r"../data\NACRES_embedded.pkl")

In [21]:
#merge df_NACRES with df_gpt_source on "Chosen_NACRES_name" and "nacres.description.en"
df_gpt_source_NACRES = pd.merge(df_gpt_source, df_NACRES, left_on="Choosen option", right_on="nacres.description.en", how="left")

In [23]:
df_gpt_source_NACRES.to_excel(r"../data\results/achats_GVG_NACRES_GPT.xlsx")

In [24]:
df_gpt_source_NACRES.columns

Index(['Article name', 'Category', 'Provider', 'Options', 'Choosen option',
       'Référence fournisseur', 'Désignation article', 'Package Size',
       'Package Unit', 'N° Commande', 'Unnamed: 5', 'Date de création',
       'N° Fournisseur', 'Fournisseur', 'Prix unitaire', 'Prix total', 'Unité',
       'Quantité Commandé', 'Unité<br>commande', 'Libellé FR Centre de coût',
       'Code Centre de coût', 'Code', 'Libellé', 'Famille', 'ID de famille',
       'lvl_code', 'Famille1', 'Libellé FR Domaine', 'Libellé FR Famille',
       'Libellé FR Sous-sous-famille', 'Famille_libellé anglais', 'combined',
       'embedding_x', 'nacres.code', 'nacres.description.fr',
       'nacres.description.en', 'ceda.code', 'ceda.description',
       'ceda.ef.kg.co2e.per.euro', 'ceda.uncertainty.attr.kg.co2e.per.euro',
       'ceda.uncertainty.80pct.kg.co2e.per.euro', 'ademe.code',
       'ademe.description', 'ademe.ef.kg.co2e.per.euro',
       'ademe.uncertainty.attr.kg.co2e.per.euro',
       'ademe.unce